# Bayesian A/B Testing

Kaggle: [https://www.kaggle.com/code/andrewbrdk/bayesian-a-b-testing](https://www.kaggle.com/code/andrewbrdk/bayesian-a-b-testing)  

GitHub: [https://github.com/andrewbrdk/Bayesian-AB-Testing](https://github.com/andrewbrdk/Bayesian-AB-Testing) 

*A/B testing is reviewed, and Bayesian modelling is discussed. Conversions, means, transactional revenue per user, and orders per visitor are compared in Bayesian approach.*

&nbsp; &nbsp; *- [A/B Tests](#A/B-Tests)*  
&nbsp; &nbsp; *- [Bayesian Modelling](#Bayesian-Modelling)*  
&nbsp; &nbsp; *- [Conversions](#Conversions)*   
&nbsp; &nbsp; *- [Means](#Means)*    
&nbsp; &nbsp; *- [Transactional Revenue per User](#Transactional-Revenue-per-User)*  
&nbsp; &nbsp; *- [Orders per Visitor](#Orders-per-Visitor)*  
&nbsp; &nbsp; *- [Conclusion](#Conclusion)*  
&nbsp; &nbsp; *- [References](#References)* 

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from collections import namedtuple
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
plt.rcParams.update({
    "figure.facecolor": "white",
    "axes.facecolor": "white",
    "axes.grid": True,
    "grid.color": "whitesmoke",
    "axes.axisbelow": True,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.titlesize": 14,
    "axes.titlepad": 20,
    "axes.titlelocation": "left",
    "lines.linewidth": 1.3,
    "legend.frameon": False,
    "figure.figsize": (10, 5),
    "figure.dpi": 200,
    "savefig.dpi": 200,
})
np.random.seed(7)

# A/B Tests

Mobile apps and web services are constantly updated to drive revenue, conversions, engagement, and other key metrics. Some updates can hurt the product. It is essential to measure the impact of new features.

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_experiment_versions.png?raw=true" alt="experiment_versions" width="400"/>
    
<em>Raising prices (on the right) would increase average order value but drop conversion. The net effect on revenue is unpredictable and should be measured. </em>
</center>

Comparing metrics before and after a release doesn't show the true impact. Other factors, such as marketing campaigns, also affect them. Changes in metrics over time can't be attributed to a particular release unless the effect is strong enough to stand out over natural fluctuations.

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_effect_size.png?raw=true" alt="effect_size"  width="900"/>
<em>Metrics dynamics is influenced by numerous factors. Unless the feature impact is strong enough (e.g., a sharp 30% drop), a change is hard to attribute to a particular release. </em>
</center>

A/B tests isolate the impact of the new feature. The original and modified versions are launched in parallel. Users are randomly split between the variants. Key metrics are compared in each group. The test is stopped when one version clearly outperforms the other or the experiment has been running long enough with no benefits to continue. The version to roll out for all users is decided upon the results.

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_ab_test.png?raw=true" alt="ab_test" width="800"/>
    
<em>A/B experiment setup: the test versions run in parallel. Users are randomly split between the variants. Next steps are determined upon key metrics comparison between the groups. </em>
</center>

The A/B testing causal diagram [[CausalDAG](https://en.wikipedia.org/wiki/Causal_graph)] is as follows. Metrics are determined by users' actions in the service. The actions depend on the version of the website or app (e.g., available pricing plans), external factors (e.g., seasonality), and user segments (e.g., new vs. returning customers). In an A/B test the versions run simultaneously. External factors influence both groups equally. Users are split between the groups randomly, so the segment composition is identical. Differences in metrics between the groups are attributed to the new features.

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_causal.png?raw=true" alt="causal" width="600"/>
    
<em>Metrics are determined by users' actions in the service. The actions depend on the current version of the service, external factors, and user segments. In A/B test variants run simultaneously, so external factors impact metrics equally. Random user assignment ensures segments are comparable. As a result, differences in metrics between groups are attributed to the tested functionality. </em>
</center>

In A/B tests it is necessary to estimate metrics in each group, the impact of the feature, and select the "best" variant. The exact metric values are unknown and estimated from the collected data. Metrics can be treated as random variables. Probability distributions for these variables should fit the experimental data. Comparing the distributions allows to assess the effect. It’s convenient to present point estimates and highest probability density intervals. For example, a metric in Group A is $p_A = 7.1 \pm 0.2$, and in Group B is $p_B = 7.4 \pm 0.3$. The relative difference shows the effect $(p_B - p_A) / p_A = 4.2 \pm 0.2\%$. To select the "best" group the probability metric in B is higher than A is evaluated $P(p_B > p_A) = 95\%$. Probability is understood in the subjective sense, as a measure of confidence in a particular outcome among several possibilities [[SubjProb](https://en.wikipedia.org/wiki/Probability_interpretations#Subjectivism)].

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_ab_metric_random.png?raw=true" alt="ab_metric_random" width="500"/>
<br/>   
<em>
In A/B tests it is necessary to evaluate metrics, effects, and choose the "best" group. Exact metric values are unknown and estimated from the collected data. The estimates are treated as random variables. The probability distributions for these variables should fit the experimental data. Comparing the distributions allows to asses the effect.
</em>
</center>

Metric estimates are imprecise when data is limited. Estimates improve as more data is gathered. Confidence in better group also grows. The experiment can be stopped when confidence reaches a sufficient level. However, different stopping criteria may be employed.

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_ab_dynamics.png?raw=true" alt="ab_dynamics" width="430"/>
<br/>
<em>
As data accumulates, the accuracy of metric estimates improves, and confidence in identifying the better group grows. The experiment can be stopped once confidence reaches a sufficient level.
</em>
</center>

Distributions of metrics based on experimental data are estimated with Bayesian modelling [[SR](https://www.routledge.com/Statistical-Rethinking-A-Bayesian-Course-with-Examples-in-R-and-STAN/McElreath/p/book/9780367139919), [SGBS](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364)].

# Bayesian Modelling

The first example. Will it rain during the day if it's cloudy in the morning? To answer this, we can calculate the probability of rain given that it's cloudy, $P(\mbox{Rain | Clouds}) = (\mbox{Clouds, Rain}) / (\mbox{Clouds})$. The total number of cloudy days is the sum of cloudy days with rain and cloudy days without rain, $(\mbox{Clouds}) = \mbox{(Clouds, Rain)} + \mbox{(Clouds, No Rain)}$. Let’s assume 20% of days are rainy, $P(\text{Rain}) = 20\%$, the probability of morning cloudiness on a rainy day is $P(\mbox{Clouds | Rain}) = 70\%$, and on a dry day, $P(\mbox{Clouds | No Rain}) = 10\%$. The number of cloudy days with rain can be expressed using these probabilities: $\mbox{(Clouds, Rain)} = (\mbox{Total Days}) P(\mbox{Rain}) P(\mbox{Clouds | Rain})$. Similarly, the number of cloudy days without rain can be calculated. After substituting values, we get $P(\mbox{Rain | Clouds}) = (0.7 \cdot 0.2) / (0.7 \cdot 0.2 + 0.1 \cdot 0.8) = 63.6\%$.

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_bayes_rain.png?raw=true" alt="bayes_rain" width="600"/>
<br/>
<em>
The probability of a rainy day given a cloudy morning is estimated by the ratio of rainy, cloudy days to all cloudy days — both with and without rain.
</em>
</center>

$$
\begin{split}
P(\mbox{Rain | Clouds}) & = \frac{\mbox{Clouds, Rain}}{\mbox{Clouds, Rain + Clouds, No Rain}} 
\\
\\
& = \frac{P(\mbox{Clouds | Rain})P(\mbox{Rain})}{P(\mbox{Clouds | Rain})P(\mbox{Rain}) + P(\mbox{Clouds | No Rain})P(\mbox{No Rain})}
\\
\\
& = \frac{0.7 \cdot 0.2}{0.7 \cdot 0.2 + 0.1 \cdot 0.8} = 63.6 \%
\end{split}
$$

In estimating the probability of rain given cloudy weather, $P(\text{Rain | Clouds})$, it's important to consider not only the probability of cloudiness on a rainy day, $P(\text{Clouds | Rain})$, but also the proportion of rainy days, $P(\text{Rain})$, and the probability of cloudiness on a dry day, $P(\text{Clouds | No Rain})$. Ignoring these factors can lead to a base rate fallacy [[BaseFal](https://en.wikipedia.org/wiki/Base_rate_fallacy)].

Another example. In the evening, you see an unfamiliar object in the park. From a distance, only its outline is visible. You try to guess what it is. Formally, this is a Bayes’ rule problem. You list possible options — leaves, a lost hat, a bird, a puddle. For each, you estimate how likely it is to match the observed shape: $P(\mbox{Shape | Leaves})P(\mbox{Shape | Leaves})$, $P(\mbox{Shape | Hat})P(\mbox{Shape | Hat})$, etc. You also factor in how common each option is — leaves are much more likely than a lost hat: $P(\mbox{Leaves})>P(\mbox{Hat})$. Bayes’ rule combines this information to estimate the probability of each option given what you see: $P(\mbox{Leaves | Shape}) \propto P(\mbox{Shape | Leaves}) P(\mbox{Leaves})$. As you get closer, the object glances at you and quickly climbs a tree — it turns out to be a squirrel you haven’t seen in the park for a while.

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_bayes_park.png?raw=true" alt="park" width="600"/>
<em>
<br/>
To choose between options using Bayes' rule, you need to account for two things: how common each option is (the width of the vertical bars) and how likely it is to produce the observed shape (the height of the shaded area within the bar). The probability of each option is proportional to the area of its shaded region relative to the total shaded area across all options.
</em>
</center>

This example illustrates the key elements of Bayesian modeling. We have data or observations $\mathcal{D}$ and possible explanations or hypotheses $\mathcal{H}_i$. To choose between them, we evaluate how well each hypothesis explains the data — this is done by calculating likelihoods $P(\mathcal{D}|\mathcal{H}_i)$. Prior knowledge or experience is reflected in the prior probabilities $P(\mathcal{H}_i)$. Bayes' rule combines the likelihood and the prior to compute the posterior probability $P(\mathcal{H}_i|\mathcal{D})$ — our updated belief in each hypothesis given the data. Based on these posteriors, we select the most suitable model. It's important to validate models: even if one hypothesis fits the data better than the others, none of them may fully reflect reality.

$$
\begin{split}
P(\mathcal{H}_i | \mathcal{D}) &= \frac{ P(\mathcal{D} | \mathcal{H}_i) P(\mathcal{H}_i) }{P(\mathcal{D})}
= \frac{ P(\mathcal{D} | \mathcal{H}_i) P(\mathcal{H}_i) }{\sum \limits_i P(\mathcal{D} | \mathcal{H}_i) P(\mathcal{H}_i) }
\\
P(\mathcal{H}_i | \mathcal{D}) &\mbox{ - posterior probability distribution} 
\\
P(\mathcal{D} | \mathcal{H}_i) &\mbox{ - likelihood function}
\\
P(\mathcal{H}_i) &\mbox{ - prior probability distribution}
\\
P(\mathcal{D}) &\mbox{ - normalization constant}
\end{split}
$$

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_bayes_hypotheses_square.png?raw=true" alt="bayes_hypotheses_square" width="900"/>
<em>
<br/>
A set of models is selected to explain the data. For each model, we specify prior belief — its probability relative to other models. We then calculate the likelihood — the probability of observing the data assuming the model is true. Using Bayes' rule, we update our belief in each model given the data — this gives the posterior probability.
</em>
</center>

Next example. Suppose $N=1000$ users visited a webpage, and $n_s=100$ of them clicked the “Continue” button. What does the distribution of the possible conversion rate $p$ look like? We assume each user has the same probability of converting, and — before seeing the data — all possible values of $p$ are considered equally likely.

We need to estimate the probability $P(\mathcal{H} | \mathcal{D}) = P(p | n_s, N)$ for given $n_s$ и $N$. Using Bayes’ rule, we have: $P(p | n_s, N) \propto P(n_s, N | p) P(p)$. Each user clicks with probability $p$ and does not click with probability $1-p$. The clicks of $N$ users can be modeled as a sequence of Bernoulli random variables [[BernProc](https://en.wikipedia.org/wiki/Bernoulli_process), [SciPyBern](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bernoulli.html)]. The probability of $n_s$ conversions out of $N$ with success probability $p$ is given by a binomial distribution $P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N; p)$ [[BinomDist](https://en.wikipedia.org/wiki/Binomial_distribution), [SciPyBinom](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html)]. Since all possible prior values of the conversion rate are equally likely, the prior distribution is uniform $P(\mathcal{H}) = P(p) = \mbox{Unif}(0, 1) = 1$. The posterior distribution $P(p | n_s, N)$ will be a Beta distribution [[BetaDist](https://en.wikipedia.org/wiki/Beta_distribution), [SciPyBeta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html)].

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) & = \mbox{Binom}(n_s, N; p) = C^{n_s}_{N} p^{n_s} (1 - p)^{N-n_s}
\\
\\
P(\mathcal{H}) = P(p) & = \mbox{Unif}(0, 1) = 1
\\
\\
P(\mathcal{H} | \mathcal{D}) = P(p | n_s, N) 
& = \frac{P(n_s, N | p) P(p)}{P(n_s, N)}
= \frac{P(n_s, N | p) P(p)}{\int_0^1 d p P(n_s, N | p) P(p)}
\\
& = \frac{p^{n_s} (1 - p)^{N-n_s}}{\int_0^1 d p (1 - p)^{N-n_s} p^{n_s} }
= \mbox{Beta}(p; n_s + 1, N - n_s + 1)
\\
\\
\mbox{Beta}(x; \alpha, \beta) & \equiv \frac{x^{\alpha-1} (1 - x)^{\beta-1}}{\int_0^1 dx x^{\alpha-1} (1 - x)^{\beta-1}}
 = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1} (1 - x)^{\beta-1}
\end{split}
$$

The graph of the posterior distribution $ P(p | n_s, N) $ is shown below. The mode coincides with the sample mean $n_s/N$, and the most likely values of $p$ are near this value.

In [ ]:
ns = 100
ntotal = 1000

p_samp = ns / ntotal
p_dist = stats.beta(a=ns+1, b=ntotal-ns+1)

xaxis_max = 0.2
x = np.linspace(0, xaxis_max, 1000)
y = p_dist.pdf(x) 
plt.figure()
plt.plot(x, y, color="black", label="Distribution")
plt.vlines(p_samp, ymin=0, ymax=y.max(),
    colors="black", linestyles="dashed", label="Sample mean"
)
plt.title(r"Posterior Distribution $P(p \mid n_s, N)$")
plt.xlabel("$p$")
plt.ylabel("Probability density")
plt.xlim(0, xaxis_max)
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
plt.legend()
plt.tight_layout()
plt.show()

Another example. For version A of a webpage, $N_A=1000$ users visited, and $n_{s_A}=100$ clicked the "Continue" button. For version B, $N_B=1000$ users visited, and $n_{s_B}=110$ clicked the "Continue" button. What is the probability that the conversion rate for page B is higher than for page A?

You need the probability $P(p_B > p_A)$. The posterior distribution of the conversion rates for each group is calculated as in the previous example: $P(p; n_s, N) = \mbox{Beta}(p; n_s + 1, N - n_s + 1)$. To estimate $P(p_B > p_A)$, we can calculate the distribution of $p_B - p_A$ and find the probability $P(p_B - p_A > 0)$. The distribution of $p_B - p_A$ in general is given by the convolution of the two distributions [[ProbConv](https://en.wikipedia.org/wiki/Convolution_of_probability_distributions)]. In this case, we can use an approximation. Given the parameters, the posterior Beta distributions are close to normal distributions [[BetaDist](https://en.wikipedia.org/wiki/Beta_distribution#Special_and_limiting_cases), [NormDist](https://en.wikipedia.org/wiki/Normal_distribution), [SciPyNorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html?highlight=norm)]. The difference of random variables with normal distributions is also a random variable with a normal distribution [[NormSum](https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables)]. Therefore, the distribution of $p_B - p_A$ is approximately normal, with a mean equal to the difference of the means of the posterior Beta distributions and a variance equal to the sum of the variances. The desired probability is expressed using the cumulative distribution function $P(p_B - p_A > 0) = 1 - F(0)$ [[CDF](https://en.wikipedia.org/wiki/Cumulative_distribution_function)]. Instead of analytical transformations, a numerical estimate can be performed. For this, samples are generated from the posterior distributions and compared. The first graph shows the posterior distributions in the groups. The second shows the approximate analytical distribution of $p_B - p_A$ and the distribution of the difference between samples from the posterior distributions. Both calculations yield a similar result: $P(p_B > p_A) = 77 \%$.

$$
\begin{split}
P(p_B > p_A) & = P(p_B - p_A > 0)
\\
\\
P_{p_A}(x) = \mbox{Beta}(x; n_{s_A} + 1, N_A - n_{s_A} + 1)
& \approx \mbox{Norm}(x; \mu_A, \sigma_A^2),
\quad
\mu_A = n_{s_A}/N_A, 
\,
\sigma_A^2 = \mu_A (1 - \mu_A) / N_A,
\quad
N_A \gg n_{s_A} \gg 1
\\
\\
P_{p_B}(x) = \mbox{Beta}(x; n_{s_B} + 1, N_B - n_{s_B} + 1)
& \approx \mbox{Norm}(x; \mu_B, \sigma_B^2),
\quad
\mu_B = n_{s_B}/N_B, 
\,
\sigma_B^2 = \mu_B (1 - \mu_B) / N_B,
\quad
N_B \gg n_{s_B} \gg 1
\\
\\
P_{p_B - p_A}(x) = 
\int_{-\infty}^{\infty} dy P_{p_B}(y) P_{p_A}(y-x)
& \approx \mbox{Norm}\left(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2\right),
\quad
\mbox{Norm}(x ; \mu, \sigma^2) \equiv \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
\\
\\
P(p_B - p_A > 0) & = 1 - F_{p_B - p_A}(0)
\end{split}
$$

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
dist_p_b_gt_a = 1 - approx_diff_dist.cdf(0)

npost = 50000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_p_b_gt_a = np.sum(samp_b > samp_a) / npost

xaxis_max = 0.2
x = np.linspace(0, xaxis_max, 1000)
plt.figure()
plt.plot(x, p_dist_a.pdf(x), color="black", label="A")
plt.plot(x, p_dist_b.pdf(x), color="black", alpha=0.3, label="B")
plt.title("Posterior Distributions")
plt.xlabel("$p$")
plt.ylabel("Probability density")
plt.xlim(0, xaxis_max)
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
plt.legend()
plt.tight_layout()
plt.show()

x = np.linspace(-0.3, 0.3, 1000)
plt.figure()
plt.plot(x, approx_diff_dist.pdf(x), color="black", label="Analytical approximation")
plt.hist(samp_b - samp_a, bins=500, density=True, color="black", alpha=0.3, label="Posterior samples difference")
plt.vlines(0, 0, approx_diff_dist.pdf(x).max() * 1.05, colors="black", linestyles="dashed")
plt.title("$p_B - p_A$")
plt.xlabel("$x$")
plt.ylabel("Probability density")
plt.xlim(-0.1, 0.1)
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
plt.legend()
plt.tight_layout()
plt.show()

print(f"P(p_b > p_a) diff dist: {dist_p_b_gt_a:.4f}")
print(f"P(p_b > p_a) post samples: {samp_p_b_gt_a:.4f}")

# Conversions

In A/B tests, conversions to orders, clicks on buttons, and other actions are often compared. If a user's behavior does not affect others, a Bernoulli process can be used for modeling. With a conversion rate $p$, the probability that $n_s$ out of $N$ users will perform the target action follows a binomial distribution $P(n_s, N | p) = \mbox{Binom}(n_s, N | p)$. The prior distribution for conversions is conveniently modeled with a Beta distribution $P(p) = \mbox{Beta}(p; \alpha, \beta)$. The Beta distribution's dependency on $p$, without normalization constants, is $\mbox{Beta}(p; \alpha, \beta) \propto p^{\alpha-1}(1-p)^{\beta-1}$. This form remains valid for the product of the likelihood and the prior distribution $P(p | n_s, N) \propto \mbox{Binom}(p, N) \mbox{Beta}(p; \alpha, \beta) \propto p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}$. The only important part is the dependency on $p$; the other factors will be included in the normalization constant. Therefore, the posterior distribution will also be a Beta distribution, but with different parameters $P(p | n_s, N) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)$. Prior distributions with this property are called conjugate priors [[ConjPrior](https://en.wikipedia.org/wiki/Conjugate_prior)].

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta) = 
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
\begin{split}
P(\mathcal{H} | \mathcal{D}) & = P(p | n_s, N) 
\\
& \propto \mbox{Binom}(n_s, N | p) \mbox{Beta}(p; \alpha, \beta)
\\
& \propto C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
\\
& \propto p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}
\\
& = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
\end{split}
$$

Beta distributions for different parameters are shown in the graph below [[BetaDist](https://en.wikipedia.org/wiki/Beta_distribution), [SciPyBeta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html)]. When $\alpha = 1, \beta=1$, the distribution is uniform—these values are convenient for use as prior distributions. In other cases, the maximum of the distribution occurs at $p = (\alpha-1) / (\alpha + \beta - 2)$. As $\alpha$ and $\beta$ increase, the distribution narrows and approaches a normal distribution. Instead of using $\alpha = 1, \beta=1$ for the prior, the initial values of $\alpha$ and $\beta$ can be chosen based on historical data to make the prior distribution of conversions match the historical value.

In [ ]:
x = np.linspace(0, 1, 1000)

plt.figure()
plt.plot(x, stats.beta.pdf(x, a=1, b=1), color='black', linestyle='dashed')
plt.plot(x, stats.beta.pdf(x, a=1, b=5), color='black', linestyle='solid', linewidth=1.2)
plt.plot(x, stats.beta.pdf(x, a=7, b=12), color='black', linestyle='solid', linewidth=1.2)
plt.plot(x, stats.beta.pdf(x, a=150, b=50), color='black', linestyle='solid', linewidth=1.2)

text_x = [0.93, 0.08, 0.37, 0.84]
text_y = [1.35, 5.00, 4.10, 12.0]
labels = ["a=1, b=1", "a=1, b=5", "a=7, b=12", "a=150, b=50"]
for tx, ty, label in zip(text_x, text_y, labels):
    plt.text(tx, ty, label, ha='center', va='center', fontsize=9)

plt.title("Beta Distribution Beta(a, b)")
plt.xlabel("x")
plt.ylabel("Probability density")
plt.xlim(0, 1)
plt.show()

To verify the conversion calculation using the data, we set an exact value for the conversion `p`, then generate `nsample` binary random variables. From this sample, we build the posterior distribution of possible conversion values, denoted as `post_dist`. On the graph, the mode of the posterior distribution coincides with the sample mean and is close to the true value of `p`.

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)
    
p = 0.1
nsample = 1010

exact_dist = stats.bernoulli(p=p)
data = exact_dist.rvs(nsample)
post_dist = posterior_dist_binom(ns=np.sum(data), ntotal=len(data))

x = np.linspace(0, 1, 1000)
plt.figure()
y = post_dist.pdf(x)
plt.plot(x, y, color='black', label='Posterior')
plt.plot([data.mean(), data.mean()], [0, y.max()], 
          color='black', linestyle='dashed', label='Sample mean')
plt.plot([exact_dist.mean(), exact_dist.mean()], [0, y.max() * 1.05], 
          color='red', linestyle='dashed', label='Exact p')
plt.title("Posterior Distribution")
plt.xlabel("p")
plt.ylabel("Probability density")
plt.xlim(p - 0.1, p + 0.1)
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
plt.legend()
plt.show()

For the example comparing two groups, $p_A$ is set, and $p_B$ is 5% higher. `nsample` data points are generated for each group, and posterior distributions of the conversions are built. By sampling from these distributions, we estimate the probability that group B's conversion exceeds group A's, $P(p_B > p_A)$. The graph shows $P(p_B > p_A) = 84.0 \%$. Since nsample is relatively small, the values may change with each run.

In [ ]:
def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

p_A = 0.1
p_B = p_A * 1.05
nsample = 1000

exact_dist_A = stats.bernoulli(p=p_A)
exact_dist_B = stats.bernoulli(p=p_B)
data_A = exact_dist_A.rvs(nsample)
data_B = exact_dist_B.rvs(nsample)

post_dist_A = posterior_dist_binom(ns=np.sum(data_A), ntotal=len(data_A))
post_dist_B = posterior_dist_binom(ns=np.sum(data_B), ntotal=len(data_B))

x = np.linspace(0, 1, 1000)
plt.figure()
yA = post_dist_A.pdf(x)
yB = post_dist_B.pdf(x)
plt.plot(x, yA, color='black', label='A')
plt.plot(x, yB, color='black', alpha=0.2, label='B')
plt.plot([exact_dist_A.mean(), exact_dist_A.mean()], [0, yA.max() * 1.05], 
          color='black', linestyle='dashed', label='Exact A')
plt.plot([exact_dist_B.mean(), exact_dist_B.mean()], [0, yA.max() * 1.05], 
          color='black', linestyle='dashed', alpha=0.2, label='Exact B')
plt.title("Posterior Distributions")
plt.xlabel("p")
plt.ylabel("Probability density")
plt.xlim(p_A - 0.1, p_A + 0.1)
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
plt.legend()
plt.show()

print(f'P(pB > pA): {prob_pb_gt_pa(post_dist_A, post_dist_B)}')

The example below shows how conversion estimates and the probability $P(p_B > p_A)$ change as more data is collected. Two groups are compared: Group A has a fixed conversion rate $p_A$, while Group B’s conversion rate $p_B$ is set 5% higher. In each group, `npoints` samples are generated over `nstep` iterations (for a total of `N = npoints * nstep` samples). At each step, posterior distributions are updated based on the accumulated data, and the probability $P(p_B > p_A)$ is calculated. The plot also shows 95% credible intervals for both groups. As the sample size grows, the intervals narrow, and the probability tends toward 1 in favor of the better-performing group.

In [ ]:
def posterior_binom_approx_95pdi(post_dist):
    lower = post_dist.ppf(0.025)
    upper = post_dist.ppf(0.975)
    return lower, upper

pa = 0.1
pb = pa * 1.05

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['npoints'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df[['pa_lower', 'pa_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sa'], r['N'])), axis=1, result_type="expand")
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df[['pb_lower', 'pb_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sb'], r['N'])), axis=1, result_type="expand")
df['pb_gt_pa'] = df.apply(lambda r: prob_pb_gt_pa(posterior_dist_binom(r['sa'], r['N']), posterior_dist_binom(r['sb'], r['N']), post_samp=10_000), axis=1)

plt.figure()
plt.plot(df["N"], df["pa"], color="black", label="A")
plt.fill_between(df["N"], df["pa_lower"], df["pa_upper"], color="black", alpha=0.2, label="A, 95% PDI")
plt.plot(df["N"], df["pb"], color="blue", label="B")
plt.fill_between(df["N"], df["pb_lower"], df["pb_upper"], color="blue", alpha=0.2, label="B, 95% PDI")
plt.title(r"$p_A, p_B$")
plt.xlabel("N")
plt.gca().yaxis.set_major_formatter(lambda x, _: f"{x:.1%}")
plt.legend()
plt.show()

plt.figure()
plt.plot(df["N"], df["pb_gt_pa"], color="black")
plt.title(r"$P(p_B > p_A)$")
plt.xlabel("N")
plt.ylim(0, 1)
plt.show()

The accuracy of identifying the better group is demonstrated as follows. Group A is assigned a conversion rate `p`. Group B is assigned a random value within $\pm 5\%$ of `p`. Data for both groups is generated in steps of `n_samp_step`. At each step, posterior distributions are updated, and the probability $P(p_B > p_A)$ is calculated. The experiment stops when either $P(p_B > p_A)$ or $P(p_A > p_B)$ reaches the stopping threshold `prob_stop = 0.95`, or when the maximum number of samples `n_samp_max` is reached. The procedure is repeated `nexps` times. The share of correctly identified better groups is calculated across all experiments. In this case, with `nexps = 100`, the correct group was identified 94 times. The resulting accuracy of 0.94 is close to the stopping threshold of `prob_stop = 0.95`.

In [ ]:
cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])

p = 0.1
nexps = 100
cmp['A'] = [p] * nexps
cmp['B'] = p * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))
cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B'] > r['A'] else 'A', axis=1)

n_samp_max = 30_000_000
n_samp_step = 10_000
prob_stop = 0.95

for i in range(nexps):
    pA = cmp.at[i, 'A']
    pB = cmp.at[i, 'B']
    exact_dist_A = stats.bernoulli(p=pA)
    exact_dist_B = stats.bernoulli(p=pB)
    n_samp_total = 0
    ns_A = 0
    ns_B = 0
    while n_samp_total < n_samp_max:
        dA = exact_dist_A.rvs(n_samp_step)
        dB = exact_dist_B.rvs(n_samp_step)
        n_samp_total += n_samp_step
        ns_A = ns_A + np.sum(dA)
        ns_B = ns_B + np.sum(dB)
        post_dist_A = posterior_dist_binom(ns=ns_A, ntotal=n_samp_total)
        post_dist_B = posterior_dist_binom(ns=ns_B, ntotal=n_samp_total)
        pb_gt_pa = prob_pb_gt_pa(post_dist_A, post_dist_B)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_dist_A.mean()
            cmp.at[i, 'B_exp'] = post_dist_B.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: nsamp {n_samp_total}, best_gr {best_gr}, P(b>a) {pb_gt_pa}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(10))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

Experiments stop when the specified confidence level is reached. Time estimates and other stopping criteria are discussed in the appendices [[Apx](https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/appendices)].

# Means

The Bayesian approach requires assumptions about the distributions of the quantities being compared. Model selection is always somewhat arbitrary, and its justification is never free of questions. In many cases, a full distribution is unnecessary — comparing means is enough: average revenue per user, average session duration, and so on. For means, the Central Limit Theorem often applies [[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)]. This allows the normal distribution [[NormDist](https://en.wikipedia.org/wiki/Normal_distribution), [SciPyNorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html?highlight=norm)] to be used as a likelihood function, even when the shape of the original distribution is unknown.

The Central Limit Theorem formalizes a simple observation. Take any distribution with mean $\mu$ and variance $\sigma^2$. Draw samples of size $N$ and calculate the mean of each sample. The resulting sample means will follow an approximately normal distribution: $Norm(\mu, \sigma^2/N)$.

<center>
<img src="https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/figs/en_central_limit_theorem.png?raw=true" alt="Central Limit Theorem" width="800"/>
<br>
<em>
    Sample means from any distribution with a finite mean and variance are approximately normally distributed.
</em>
</center>

There are several versions of the central limit theorem [[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)]. One is as follows. Let $X_1, X_2, \dots$ be a sequence of independent and identically distributed random variables with a finite mean $\mu$ and variance $\sigma^2$. Let $\overline{X}_N = \frac{1}{N} \sum_{i=1}^{N} X_i$ be the sample mean. As $N$ increases, the distribution of the centered and scaled sample means converges to a normal distribution with mean 0 and variance 1. Convergence here refers to convergence in distribution [[RandVarsConv](https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_distribution)].

$$
\begin{split}
P \left( \frac{\overline{X}_N - \mu}{\sigma / \sqrt{N}} = x \right) & \to Norm(x; 0, 1), \quad N \to \infty
\\ 
Norm(x ; \mu, \sigma^2) & = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
\end{split}
$$

The graph compares the distribution of sample means with the normal distribution based on the central limit theorem. A gamma distribution $P(x; \alpha, \beta) \propto x^{\alpha-1} \exp(-\beta x)$ [[GammaDist](https://en.wikipedia.org/wiki/Gamma_distribution), [SciPyGamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html)] is used to generate `n_sample` samples, each containing `sample_len` points. The means of these samples are computed, and their distribution is plotted. Using the exact mean and variance of the original distribution, the parameters for the normal distribution based on the central limit theorem `clt_mu, clt_stdev` are calculated. This distribution is also shown on the graph. Visually, the distribution of the sample means closely approximates the normal distribution.

In [ ]:
a = 1
sample_len = 100
n_samples = 3000

exact_dist = stats.gamma(a=a)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std()

x = np.linspace(0, 10, 1000)
plt.figure()
plt.plot(x, exact_dist.pdf(x), color='black', linestyle='solid', label='Original distribution')
plt.hist(np.concatenate(samp), bins=500, density=True, color='black', alpha=0.3, label='Sample')
plt.plot(x, stats.norm.pdf(x, loc=clt_mu, scale=clt_stdev), color='black', linestyle='dashed', label=r'$\mathrm{Norm}(\mu, \sigma^2/N)$')
plt.hist(means, bins=50, density=True, color='green', alpha=0.5, label='Sample means')
plt.title("Sample Means")
plt.xlabel("x")
plt.ylabel("Probability density")
plt.xlim(0, 5)
plt.legend()
plt.show()


The central limit theorem states that the distribution of centered and scaled sample means $\overline{X}_N$ converges to a normal distribution as $N$ approaches infinity. For finite $N$, normality is not guaranteed. The deviation from normality for a finite number of terms is explained by the Berry-Esseen theorem [[BerryEsseenTheorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)]. The difference depends on $N$, the variance, and the skewness of the original distribution. The central limit theorem assumes the original distribution has finite mean and variance. Distributions that may not meet these conditions include the Pareto distribution [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)] and the Lomax distribution [[LomaxDist](https://en.wikipedia.org/wiki/Lomax_distribution), [SciPyLomax](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lomax.html)]. The probability density function of the Lomax distribution is as follows:

$$
P(x; c) = \frac{c}{(1 + x )^{c + 1}}, \quad x \ge 0, c > 0.
$$

For values of the parameter $c$ less than or equal to 2, the variance of the Lomax distribution is infinite.
The histograms below show the distribution of `n_samples` sample means, each with `sample_len` terms, along with a normal distribution based on the central limit theorem parameters `clt_mu,clt_stdev`. The distribution of sample means is skewed and deviates more from the normal distribution than in the previous case. This skewness arises from the inclusion of large values from the tail of the original distribution in the samples. In practice, distributions are bounded, so variances and means are finite. The central limit theorem still applies, but a large number of points will be needed to approximate sample means with a normal distribution.

In [ ]:
c = 1.7
sample_len = 500
n_samples = 1000

exact_dist = stats.lomax(c=c)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std()

xaxis_max=10
x = np.linspace(0, xaxis_max, 2000)
plt.figure()
plt.plot(x, exact_dist.pdf(x), color='black', linestyle='solid', label='Original distribution')
plt.hist(np.concatenate(samp)[np.concatenate(samp) < xaxis_max], 
         bins=500, density=True, color='black', alpha=0.3, label='Sample')
plt.plot(x, stats.norm.pdf(x, loc=clt_mu, scale=means_stdev), 
         color='black', linestyle='dashed', label=r'$\mathrm{Norm}(\mu, \sigma_{\overline{X}_N}^2)$')
plt.hist(means, bins=150, density=True, color='green', alpha=0.5, label='Sample means')
plt.title("Sample Means")
plt.xlabel("x")
plt.ylabel("Probability density")
plt.xlim(0, xaxis_max)
plt.legend()
plt.show()

For Bayesian estimation of the parameters of a normal distribution based on a sample, the likelihood function is given by $P(\mathcal{D} | \mathcal{H}) = Norm(x | \mu, \sigma_x^2)$. This function has two parameters: $\mu$ and $\sigma_x$. For this model, there exists a conjugate prior distribution [[ConjPrior](https://en.wikipedia.org/wiki/Conjugate_prior), [Apx](https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/appendices)]. Below is a simplified version of this model is used where only $\mu$ is adjusted, and $\sigma_x$ is fixed. The conjugate prior distribution for $\mu$ is normal $P(\mu) = Norm(\mu | \mu_0, \sigma_0^2)$, with parameters $\mu_0$ and $\sigma_0$. To calculate the posterior distribution, multiply the likelihood functions for all data points $x_i$: $P(\mathcal{H} | \mathcal{D}) \propto \prod_i^N Norm(x_i | \mu, \sigma_x^2) Norm(\mu | \mu_0, \sigma_0^2)$. Only the dependence on $\mu$ matters in the transformations, as the terms without $\mu$ will contribute to the normalization constant. The posterior distribution remains normal: $P(\mu | \mathcal{D}) = Norm(\mu | \mu_N, \sigma_N^2)$ with updated parameters $\mu_N$ and $\sigma_N$.

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = Norm(x | \mu, \sigma_x^2) = 
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\tfrac{(x - \mu)^2}{2 \sigma_x^2}}
\\
P(\mathcal{H}) & = Norm(\mu | \mu_0, \sigma_0^2) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
\\
P(\mathcal{H} | \mathcal{D}) 
& \propto
\prod_i^N
Norm(x_i | \mu, \sigma_x^2)
Norm(\mu | \mu_0, \sigma_0^2)
\\
& \propto_{\mu}
\prod_i^N
e^{-\tfrac{(x_i - \mu)^2}{2 \sigma_x^2}}
e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_0^2}} 
\\
& \propto_{\mu}
e^{-\mu^2 \left[\tfrac{N}{2 \sigma_x^2} + \tfrac{1}{2 \sigma_0^2} \right] + 
   2\mu \left[\tfrac{\mu_0}{2 \sigma_0^2} + \tfrac{1}{2 \sigma_x^2} \sum_i^N x_i \right]}
\\
& \propto_{\mu}
e^{-\tfrac{(\mu - \mu_N)^2}{2 \sigma_N^2}}
= Norm(\mu | \mu_N, \sigma_N^2),
\quad
\sigma_N^2 = \frac{\sigma_0^2 \sigma_x^2}{\sigma_x^2 + N \sigma_0^2},
\quad
\mu_N = \mu_0 \frac{\sigma_N^2}{\sigma_0^2} + \frac{\sigma_N^2}{\sigma_x^2} \sum_i^N x_i
\end{split}
$$

To verify the construction of the posterior distribution from data, a normal distribution with parameters `mu` and `sigma` is defined, and a sample of size `nsample` is generated. The initial parameters $\sigma_x, \sigma_0$ are set to the sample’s standard deviation; $\mu_0$ is set to the first data point. The posterior parameters $\mu_N, \sigma_N$ are computed using the remaining data. Using the full dataset to define prior parameters is incorrect — it's better to use a subset or historical data. The first plot compares the posterior distribution of $\mu$ to the true mean. The mode is close to both the sample mean and the true mean. The second plot compares the posterior predictive distribution of $x$ to the original distribution. To build this distribution, sample $\mu \sim Norm(\mu | \mu_N, \sigma_N^2)$, then sample $x \sim Norm(x | \mu, \sigma_x^2)$. The resulting histogram of $x$ visually matches the original normal distribution. The final plot compares the distributions of $x$ and $\mu$. These are fundamentally different: $P(x|\mathcal{D})$ approximates the original distribution, while $P(\mu|\mathcal{D})$ reflects uncertainty in the mean estimate. The distribution of $\mu$ is significantly narrower, with variance $\sigma_N$, while the variance of $P(x|\mathcal{D})$ depends on both $\sigma_x$ and the variation in $\mu$.

In [ ]:
ConjugateNormalParams = namedtuple('ConjugateNormalParams', 'mu sigma sx')

def initial_params_normal(mu, sigma, sx):
    return ConjugateNormalParams(mu=mu, sigma=sigma, sx=sx)

def posterior_params_normal(data, initial_pars):
    N = len(data)
    sigma_n_2 = (initial_pars.sigma**2 * initial_pars.sx**2) / (initial_pars.sx**2 + N * initial_pars.sigma**2)
    mu_n = initial_pars.mu * sigma_n_2 / initial_pars.sigma**2 + np.sum(data) * sigma_n_2 / initial_pars.sx**2    
    return ConjugateNormalParams(mu=mu_n, sigma=np.sqrt(sigma_n_2), sx=initial_pars.sx)

def posterior_mu_dist(params):
    return stats.norm(loc=params.mu, scale=params.sigma)

def posterior_rvs(params, nsamp):
    mus = stats.norm.rvs(loc=params.mu, scale=params.sigma, size=nsamp)
    return stats.norm.rvs(loc=mus, scale=params.sx, size=nsamp)

mu = 3
sigma = 1
nsample = 1000
npostsamp = 100000

exact_dist = stats.norm(loc=mu, scale=sigma)
data = exact_dist.rvs(nsample)

sx = np.std(data)
mu0 = data[0]
sigma0 = np.std(data)
pars = initial_params_normal(mu=mu0, sigma=sigma0, sx=sx)
pars = posterior_params_normal(data[1:], pars)
post_mu = posterior_mu_dist(pars)
post_samp = posterior_rvs(pars, npostsamp)

x = np.linspace(0, 10, 1000)
plt.figure()
y = post_mu.pdf(x)
plt.plot(x, y, color='black', label=r'Posterior $\mu$')
plt.plot([data.mean(), data.mean()], [0, y.max()], 
          color='black', linestyle='dashed', label='Sample mean')
plt.plot([exact_dist.mean(), exact_dist.mean()], [0, y.max() * 1.05], 
          color='red', linestyle='dashed', label='Exact mean')
plt.title(r'Posterior Distribution $\mu$')
plt.xlabel(r'$\mu$')
plt.ylabel('Probability density')
plt.xlim(2, 4)
plt.legend()
plt.show()

plt.figure()
plt.plot(x, exact_dist.pdf(x), color='red', linestyle='dashed', label='Exact distribution')
plt.hist(post_samp, bins=100, density=True, color='black', alpha=0.8, label='Posterior sample x')
plt.title("Posterior Sample x")
plt.xlabel("x")
plt.ylabel("Probability density")
plt.legend()
plt.show()

plt.figure()
plt.plot(x, post_mu.pdf(x), color='black', label=r'Posterior $\mu$')
plt.plot(x, exact_dist.pdf(x), color='red', linestyle='dashed', label='Exact distribution')
plt.hist(post_samp, bins=100, density=True, color='black', alpha=0.8, label='Posterior sample x')
plt.title(r'Distributions of $x$ and $\mu$')
plt.xlabel('x')
plt.ylabel('Probability density')
plt.xlim(0, 6)
plt.legend()
plt.show()

The estimation of a mean from an arbitrary distribution is illustrated using the gamma distribution. A sample of `nsample` points is divided into segments of `nsplit` points. The mean is computed for each segment. These sample means `means`, are assumed to follow a normal distribution and are modeled using Bayesian inference. The initial parameters, $\sigma_x$ and $\sigma_0$, are set to the standard deviation of the sample means `sx = np.std(means)`, and $\mu_0$ is set to the first sample mean `mu0 = means[0]`. The segment size `nsplit = 100` is chosen arbitrarily; the Berry–Esseen theorem can guide a more accurate choice. Using the full sample mean instead is possible, but this would leave only one point for estimating $P(\mu|\mathcal{D})$, making model validation infeasible. It's better to treat nsplit as a model hyperparameter. The first plot shows the original distribution and the distribution of the sample means, which is roughly normal. The second plot displays the posterior distribution of $\mu$, whose mode is close to the sample and true means. The third plot shows the posterior predictive distribution of the sample means. It resembles a normal distribution with parameters from the central limit theorem. As before, the posterior distribution of $\mu$ is narrower than that of the sample means. For mean comparison, focus on the distribution of $\mu$.

In [ ]:
def reshape_and_compute_means(sample, n_split):
    n_means = len(sample) // n_split
    samp_reshaped = np.reshape(sample[0 : n_means * n_split], (n_means, n_split))
    means = np.array([x.mean() for x in samp_reshaped])
    return means

def exact_clt_dist(exact_dist, n_split):
    clt_mu = exact_dist.mean()
    clt_stdev = exact_dist.std() / np.sqrt(n_split)
    return stats.norm(loc=clt_mu, scale=clt_stdev)

def sample_clt_dist(means):
    clt_mu = means.mean()
    clt_std = means.std()
    return stats.norm(loc=clt_mu, scale=clt_std)

nsample = 50000
nsplit = 100
npostsamp = 50000

a = 1
b = 2
exact_dist = stats.gamma(a=a, scale=1/b)
data = exact_dist.rvs(nsample)

means = reshape_and_compute_means(data, nsplit)
clt_dist_exact = exact_clt_dist(exact_dist, nsplit)

sx = np.std(means)
mu0 = means[0]
sigma0 = sx
pars = initial_params_normal(mu=mu0, sigma=sigma0, sx=sx)
pars = posterior_params_normal(means[1:], pars)
post_mu = posterior_mu_dist(pars)
post_samp = posterior_rvs(pars, npostsamp)

x = np.linspace(0, 10, 1000)
plt.figure()
plt.plot(x, exact_dist.pdf(x), color='black', linestyle='solid', label='Original distribution')
plt.plot(x, clt_dist_exact.pdf(x), color='black', linestyle='dashed', label=r'$Norm(\mu, \sigma^2/N)$')
plt.hist(means, bins=50, density=True, color='green', alpha=0.5, label='Sample means')
plt.title("Sample Means")
plt.xlabel("x")
plt.ylabel("Probability density")
plt.xlim(0, 3)
plt.legend()
plt.show()

x = np.linspace(0, 4, 10000)
plt.figure()
y = post_mu.pdf(x)
plt.plot(x, y, color='black', label=r'$\mu$ distribution')
plt.plot([data.mean(), data.mean()], [0, y.max()], 
          color='black', linestyle='dashed', label='Sample mean')
plt.plot([exact_dist.mean(), exact_dist.mean()], [0, y.max() * 1.05], 
          color='red', linestyle='dashed', label='Exact mean')
plt.title(r'Distribution of $\mu$')
plt.xlabel(r'$\mu$')
plt.ylabel('Probability density')
plt.xlim(exact_dist.mean() - 0.1, exact_dist.mean() + 0.1)
plt.legend()
plt.show()

plt.figure()
plt.plot(x, exact_dist.pdf(x), color='black', linestyle='solid', label='Original distribution')
plt.plot(x, clt_dist_exact.pdf(x), color='black', linestyle='dashed', label=r'$Norm(\mu, \sigma^2/N)$')
plt.hist(post_samp, bins=300, density=True, color='black', alpha=0.2, label=r'Posterior $\overline{X}_N$')
plt.title(r'Posterior Distribution $\overline{X}_N$')
plt.xlabel('x')
plt.ylabel('Probability density')
plt.xlim(0, 3)
plt.legend()
plt.show()

For the group comparison example, two gamma distributions are defined. The parameters $a$ are identical, while the parameter $b$ for group B is 5% smaller than that of group A. A sample of size nsample is drawn from each, and sample means are calculated. The posterior distributions of $\mu$ are constructed based on the sample means. From these distributions, the probability that the mean for group B is greater than for group A, $P(\mu_B > \mu_A)$, is computed. In the first plot, the original distributions and exact means are shown. In the second plot, the distributions of $\mu$ and the exact means are shown. With the chosen samples, the distributions $P(\mu|\mathcal{D})$ barely overlap, and $P(\mu_B > \mu_A) = 1$.

In [ ]:
def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

nsample = 30000
npostsamp = 50000
nsplit = 100

A, B = {}, {}
A['dist_pars'] = {'a': 1, 'b': 2}
B['dist_pars'] = {'a': 1, 'b': 2*0.95}
for g in [A, B]:
    g['exact_dist'] = stats.gamma(a=g['dist_pars']['a'], scale=1/g['dist_pars']['b'])
    g['data'] = g['exact_dist'].rvs(nsample)
    g['means'] = reshape_and_compute_means(g['data'], nsplit)
    g['post_pars'] = initial_params_normal(mu=g['means'][0], sigma=np.std(g['means']), sx=np.std(g['means']))
    g['post_pars'] = posterior_params_normal(g['means'][1:], g['post_pars'])
    g['post_mu'] = posterior_mu_dist(g['post_pars'])
    g['post_samp'] = posterior_rvs(g['post_pars'], npostsamp)

x = np.linspace(0, 3, 5000)
plt.figure()
yA = A["exact_dist"].pdf(x)
yB = B["exact_dist"].pdf(x)
plt.plot(x, yA, color="black", alpha=0.2, label="A")
plt.plot(x, yB, color="black", label="B")
plt.plot([A["exact_dist"].mean(), A["exact_dist"].mean()], [0, yA.max() * 1.05], 
          color="black", linestyle="dashed", alpha=0.2, label="Exact mean A")
plt.plot([B["exact_dist"].mean(), B["exact_dist"].mean()], [0, yB.max() * 1.05], 
          color="black", linestyle="dashed", label="Exact mean B")
plt.title("Original Distributions")
plt.xlabel("x")
plt.ylabel("Probability density")
plt.xlim(0, 3)
plt.legend()
plt.show()

plt.figure()
yA = A["post_mu"].pdf(x)
yB = B["post_mu"].pdf(x)
plt.plot(x, yA, color="black", alpha=0.2, label="A")
plt.plot(x, yB, color="black", label="B")
plt.plot([A["exact_dist"].mean(), A["exact_dist"].mean()], [0, yA.max() * 1.05],
         color="black", linestyle="dashed", alpha=0.2, label="Exact mean A")
plt.plot([B["exact_dist"].mean(), B["exact_dist"].mean()], [0, yB.max() * 1.05],
         color="black", linestyle="dashed", label="Exact mean B")
plt.title(r"$\mathrm{Distributions\ of\ }\mu$")
plt.xlabel(r"$\mu$")
plt.ylabel("Probability density")
plt.xlim(A["exact_dist"].mean() - 0.1, A["exact_dist"].mean() + 0.1)
plt.legend()
plt.show()

print(f"P(mu_B > mu_A): {prob_pb_gt_pa(A['post_mu'], B['post_mu'])}")

To demonstrate the proportion of correctly identified options, two groups with gamma distributions are set up. In group A, the parameters of the gamma distribution are fixed, while in group B, the parameter bb changes within $\pm5\%$ of group A. Along with the parameters, the means change. Data are generated from the distributions with a step size of `n_samp_step`. At each step, sample means `nsplit` are calculated. The parameters $\mu$ are estimated based on the sample means. These distributions of parameters are compared. The data collection stops when either $P(\mu_B > \mu_A)$ or $P(\mu_A > \mu_B)$ reaches `prob_stop = 0.95`, or the maximum number of points `n_samp_max` is reached. A total of `nexps` experiments are conducted, and the proportion of correctly identified groups with the larger mean is calculated. In this example, the proportion of 0.97 is close to `prob_stop = 0.95`.

In [ ]:
nexps = 100
prob_stop = 0.95
nsplit = 100
n_samp_max = 1_000_000
n_samp_step = 10_000

A = {'a': 1, 'b': 2}

cmp = pd.DataFrame(columns=['A_pars', 'B_pars', 'A_mean', 'B_mean', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])
cmp['A_pars'] = [A] * nexps
cmp['B_pars'] = cmp['A_pars'].apply(lambda x: {'a': x['a'], 'b': x['b'] * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1))})
cmp['A_mean'] = cmp['A_pars'].apply(lambda x: stats.gamma(a=x['a'], scale=1/x['b']).mean())
cmp['B_mean'] = cmp['B_pars'].apply(lambda x: stats.gamma(a=x['a'], scale=1/x['b']).mean())
cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B_mean'] > r['A_mean'] else 'A', axis=1)

for i in range(nexps):
    A_pars = cmp.at[i, 'A_pars']
    B_pars = cmp.at[i, 'B_pars']
    exact_dist_A = stats.gamma(a=A_pars['a'], scale=1/A_pars['b'])
    exact_dist_B = stats.gamma(a=B_pars['a'], scale=1/B_pars['b'])
    n_samp_total = 0
    dA = []
    dB = []
    while n_samp_total < n_samp_max:
        dA.extend(exact_dist_A.rvs(n_samp_step))
        dB.extend(exact_dist_B.rvs(n_samp_step))
        n_samp_total += n_samp_step
        means_A = reshape_and_compute_means(dA, nsplit)
        post_pars_A = initial_params_normal(mu=means_A[0], sigma=np.std(means_A), sx=np.std(means_A))
        post_pars_A = posterior_params_normal(means_A[1:], post_pars_A)
        post_mu_A = posterior_mu_dist(post_pars_A)
        means_B = reshape_and_compute_means(dB, nsplit)
        post_pars_B = initial_params_normal(mu=means_B[0], sigma=np.std(means_B), sx=np.std(means_B))
        post_pars_B = posterior_params_normal(means_B[1:], post_pars_B)
        post_mu_B = posterior_mu_dist(post_pars_B)
        pb_gt_pa = prob_pb_gt_pa(post_mu_A, post_mu_B)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_mu_A.mean()
            cmp.at[i, 'B_exp'] = post_mu_B.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: nsamp {n_samp_total}, best_gr {best_gr}, P(B>A) {pb_gt_pa}')


cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(8))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

# Transactional Revenue per User

To estimate the monetary effect, the revenue per user in the groups $P_{\text{users}}(x)$ is compared. It is useful to separate the revenue for paying users, denoted as $P_{\text{paying}}(x)$. With a conversion rate $p$, the distribution of non-zero revenue per user is $pP_{\text{paying}}(x)$, and with probability $1−p$, the revenue is zero.

$$
P_{\text{users}}(x) = 
\begin{cases}
1-p, \, x = 0
\\
p P_{\text{paying}}(x), \, x > 0
\end{cases}
$$

The conversion rate $p$ was estimated earlier. The revenue per paying user can be modeled using a log-normal distribution [[LognormDist](https://en.wikipedia.org/wiki/Log-normal_distribution),
[SciPyLognorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html)] or a Pareto distribution [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)], similar to wealth distribution. For transactional services, particularly marketplaces, the log-normal distribution is more appropriate. A random variable $X$ is log-normal $X \sim Lognormal(\mu, s^2)$ if the logarithm of $X$ is normally distributed $\ln(X) \sim Norm(\mu, s^2)$. The probability density function is provided below.

$$
\begin{split}
P(x) & = \frac{1}{x s \sqrt{2 \pi}} e^{-\tfrac{(\ln(x) - \mu)^2}{2 s^2}}
\end{split}
$$

In [ ]:
x = np.linspace(0, 20, 2000)
plt.figure()
mu, s = 1, 1
plt.plot(x,stats.lognorm.pdf(x, s=s, scale=np.exp(mu)), color="black", label=r"$\mu=1,\,s=1$")
mu, s = 2, 1
plt.plot(x,stats.lognorm.pdf(x, s=s, scale=np.exp(mu)), color="black", label=r"$\mu=2,\,s=1$")
mu, s = 1, 2
plt.plot(x,stats.lognorm.pdf(x, s=s, scale=np.exp(mu)), color="black", label=r"$\mu=1,\,s=2$")
plt.text(2.95, 0.25, r"$\mu=1,\,s=1$", ha='center', va='center', fontsize=9)
plt.text(11.2, 0.07, r"$\mu=2,\,s=1$", ha='center', va='center', fontsize=9)
plt.text(1.80, 0.48, r"$\mu=1,\,s=2$", ha='center', va='center', fontsize=9)
plt.title("Log-normal Distribution")
plt.xlabel("x")
plt.ylabel("Probability density")
plt.show()

The conjugate prior distribution for the log-normal likelihood function $P(\mathcal{D} | \mathcal{H}) = Lognorm(x | \mu, s^2)$ is constructed similarly to that of the normal distribution [[ConjPrior](https://en.wikipedia.org/wiki/Conjugate_prior)]. In a simplified model where only the parameter $\mu$ is estimated and the scale parameter $s$ is fixed, the conjugate prior for $\mu$ is a normal distribution $P(\mu) = Norm(\mu | \mu_0, \sigma_0^2)$ with parameters $\mu_0$ and $\sigma_0$. The resulting posterior is also a normal distribution $P(\mu | \mathcal{D}) = Norm(\mu | \mu_N, \sigma_N^2)$, with updated parameters $\mu_N$ and $\sigma_N$. The posterior mean $\mu_N$ is computed using the logarithms of the observed sample values.

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = Lognorm(x | \mu, s^2) = 
\frac{1}{x \sqrt{2 \pi s^2}} e^{-\tfrac{(\ln x - \mu)^2}{2 s^2}}
\\
P(\mathcal{H}) & = Norm(\mu | \mu_0, \sigma_0^2) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
\\
P(\mathcal{H} | \mathcal{D}) 
& \propto
\prod_i^N
Lognorm(x_i | \mu, s^2)
Norm(\mu | \mu_0, \sigma_0^2)
\\
& \propto_{\mu}
\prod_i^N
e^{-\tfrac{(\ln x_i - \mu)^2}{2 s^2}}
e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_0^2}} 
\\
& \propto_{\mu}
e^{-\mu^2 \left[\tfrac{N}{2 s^2} + \tfrac{1}{2 \sigma_0^2} \right] + 
   2\mu \left[\tfrac{\mu_0}{2 \sigma_0^2} + \tfrac{1}{2 s^2} \sum_i^N \ln x_i \right]}
\\
& \propto_{\mu}
e^{-\tfrac{(\mu - \mu_N)^2}{2 \sigma_N^2}}
= Norm(\mu | \mu_N, \sigma_N^2),
\quad
\sigma_N^2 = \frac{\sigma_0^2 s^2}{s^2 + N \sigma_0^2},
\quad
\mu_N = \mu_0 \frac{\sigma_N^2}{\sigma_0^2} + \frac{\sigma_N^2}{s^2} \sum_i^N \ln x_i
\end{split}
$$

To illustrate posterior inference from a log-normal distribution, a sample of size `nsample` is generated using parameters `mu` and `s`. The logarithm of the sample values is computed. The parameters $s$ and $\sigma_0$ are set equal to the standard deviation of the log-transformed sample, and $\mu_0$ is initialized as the value of the first point. The remaining points are used to compute the posterior parameters $\mu_N$ and $\sigma_N$. The posterior distribution of $\mu$ is shown in the first plot. The mean of a log-normal distribution is given by $E[x] = \exp(\mu + s^2/2)$, so $\mu + s^2/2$ should estimate the logarithm of the true mean. Since $\mu$ has normal distribution $P(\mu) = Norm(\mu_N, \sigma_N^2)$, the expression $\mu + s^2/2$ is also normally distributed as $Norm(\mu_N + s^2/2, \sigma_N^2)$. In the first plot, the mode of the distribution $\mu + s^2/2$ is close to the logarithm of the sample mean and the exact mean. The second plot compares the posterior predictive distribution of $x$ with the original distribution. The histogram of predicted $x$ values closely matches the original log-normal distribution.

$$
\begin{split}
P(x) & = Lognorm(x | \mu, s^2)
\\
E[x] & = e^{\mu + s^2/2}
\\
P(\mu) & = Norm(\mu | \mu_N, \sigma_N^2)
\\
P_{\mu + s^2/2}(y) & = Norm(y | \mu_N + s^2/2, \sigma_N^2)
\end{split}
$$

In [ ]:
ConjugateLognormalParams = namedtuple('ConjugateLognormalParams', 'mu sigma sx')

def initial_params_lognormal(mu, sigma, sx):
    return ConjugateLognormalParams(mu=mu, sigma=sigma, sx=sx)

def posterior_params_lognormal(data, initial_pars):
    N = len(data)
    lnx = np.log(data)
    sigma_n_2 = (initial_pars.sigma**2 * initial_pars.sx**2) / (initial_pars.sx**2 + N * initial_pars.sigma**2)
    mu_n = initial_pars.mu * sigma_n_2 / initial_pars.sigma**2 + np.sum(lnx) * sigma_n_2 / initial_pars.sx**2    
    return ConjugateLognormalParams(mu=mu_n, sigma=np.sqrt(sigma_n_2), sx=initial_pars.sx)

def posterior_mu_dist_lognormal(params):
    return stats.norm(loc=params.mu, scale=params.sigma)

def posterior_lognormal_rvs(params, nsamp):
    mus = stats.norm.rvs(loc=params.mu, scale=params.sigma, size=nsamp)
    return stats.lognorm.rvs(s=params.sx, scale=np.exp(mus), size=nsamp)

def posterior_mean_dist_lognormal(params):
    return stats.lognorm(scale=np.exp(params.mu + params.sx**2/2), s=params.sigma)

def posterior_ln_mean_dist_lognormal(params):
    return stats.norm(loc=params.mu + params.sx**2/2, scale=params.sigma)
    
s = 1
mu = 1.5
nsample = 1000

exact_dist = stats.lognorm(s=s, scale=np.exp(mu))
data = exact_dist.rvs(nsample)

lnx = np.log(data)
sx = np.std(lnx)
mu0 = lnx[0]
sigma0 = sx

pars = initial_params_lognormal(mu=mu0, sigma=sigma0, sx=sx)
pars = posterior_params_lognormal(data[1:], pars)
post_mu = posterior_mu_dist_lognormal(pars)
post_lnmeans = posterior_ln_mean_dist_lognormal(pars)
npostsamp = 10000
post_samp = posterior_lognormal_rvs(pars, npostsamp)

x = np.linspace(0, 4, 1000)
plt.figure()
y_mu = post_mu.pdf(x)
y_ln = post_lnmeans.pdf(x)
plt.plot(x, y_mu, color='black', label=r'$\mu$ distribution')
plt.plot(x, y_ln, color='black', alpha=0.2, label=r'$\mu + s^2/2$ distribution')
plt.plot([np.log(data.mean()), np.log(data.mean())], [0, y_mu.max()], 
          color='black', linestyle='dashed', label='Logarithm of sample mean')
plt.plot([np.log(exact_dist.mean()), np.log(exact_dist.mean())], [0, y_mu.max()*1.05], color='red', 
          linestyle='dashed', label='Logarithm of exact mean')
plt.title(r'Posterior Distributions of $\mu$ and $\mu + s^2/2$')
plt.xlabel(r'$\mu$')
plt.ylabel('Probability density')
plt.legend()
plt.show()



xaxis_max=20
x = np.linspace(0, xaxis_max, 10000)
plt.figure()
plt.plot(x, exact_dist.pdf(x), color='red', linestyle='dashed', label='Exact distribution')
plt.hist(post_samp[post_samp < xaxis_max], bins=100, density=True, color='black', alpha=0.8, label='Posterior sample')
plt.title(r'Posterior Distribution $x$')
plt.xlabel(r'$x$')
plt.ylabel('Probability density')
plt.legend()
plt.show()

To compare groups by expected revenue per paying user, use $E[x]=\exp(\mu + s^2/2)$. It's sufficient to compare $\mu + s^2/2$. This quantity is normally distributed as $Norm(\mu + s^2/2 | \mu_N, \sigma_N)$. In this example, two log-normal distributions are defined: one with parameters `s, mu`, and the other with `mu` increased by 5\%. A sample of `nsample` points is generated. Posterior distributions are constructed. The probability that the expected revenue in group B exceeds that of group A, $P(E[x]_B > E[x]_A)$, is close to 1. The first plot shows the original distributions and their exact means. The second shows the distributions $Norm(\mu + s^2/2 | \mu_N, \sigma_N)$ for each group.

In [ ]:
def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

nsample = 3000
npostsamp = 50000

A, B = {}, {}
s = 1
mu = 2
A['dist_pars'] = {'s': s, 'scale': np.exp(mu)}
B['dist_pars'] = {'s': s, 'scale': np.exp(mu * 1.05)}
for g in [A, B]:
    g['exact_dist'] = stats.lognorm(s=g['dist_pars']['s'], scale=g['dist_pars']['scale'])
    g['data'] = g['exact_dist'].rvs(nsample)
    g['post_pars'] = initial_params_lognormal(mu=np.log(g['data'])[0], sigma=np.std(np.log(g['data'])), sx=np.std(np.log(g['data'])))
    g['post_pars'] = posterior_params_lognormal(g['data'][1:], g['post_pars'])
    g['post_ln_means_dist'] = posterior_ln_mean_dist_lognormal(g['post_pars'])
    
x = np.linspace(0, 30, 1000)
plt.figure()
yA = A["exact_dist"].pdf(x)
yB = B["exact_dist"].pdf(x)
plt.plot(x, yA, color="black", alpha=0.2, label="Original A")
plt.plot(x, yB, color="black", label="Original B")
plt.plot([A["exact_dist"].mean(), A["exact_dist"].mean()], [0, yA.max() * 1.05], 
          color="red", linestyle="dashed", alpha=0.2, label="Exact mean A")
plt.plot([B["exact_dist"].mean(), B["exact_dist"].mean()], [0, yB.max() * 1.05], 
          color="red", linestyle="dashed", label="Exact mean B")
plt.title("Original Distributions")
plt.xlabel("x")
plt.ylabel("Probability density")
plt.legend()
plt.show()

x = np.linspace(0, 3, 1000)
plt.figure()
yA = A['post_ln_means_dist'].pdf(x)
yB = B['post_ln_means_dist'].pdf(x)
plt.plot(x, yA, color='black', alpha=0.2, label='A')
plt.plot(x, yB, color='black', label='B')
plt.plot([np.log(A['exact_dist'].mean()), np.log(A['exact_dist'].mean())], [0, yA.max()*1.05], 
         color='red', linestyle='dashed', alpha=0.3, label='Logarithm of exact mean A')
plt.plot([np.log(B['exact_dist'].mean()), np.log(B['exact_dist'].mean())], [0, yB.max()*1.05], 
         color='red', linestyle='dashed', label='Logarithm of exact mean B')
plt.title(r'Distribution of $\mu + s^2/2$')
plt.xlabel(r'$\mu$')
plt.ylabel('Probability density')
plt.xlim(2, 3)
plt.legend()
plt.show()

print(f"P(E[x]_B > E[x]_A): {prob_pb_gt_pa(A['post_ln_means_dist'], B['post_ln_means_dist'])}")

The share of correctly identified winning groups is evaluated for user revenue distributions, $P_{\text{users}}(x)$. Group A is assigned a fixed conversion rate $p$ and parameters $\mu, s$ for revenue per paying user. For group B, $p$ and $\mu$ are randomly varied within $\pm 5\%$ of A’s values. These parameters are changed independently, though in practice they often shift together in opposite directions. Groups are compared by expected user revenue $E_{\text{users}}[x] = p \exp(\mu + s^2/2)$. The conversion rate $p$ is estimated using a Beta distribution: $P(p) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)$, where $N$ is the total number of users, and $n_s$ is the number of paying users. Revenue per paying user is modeled with a log-normal distribution. Since $\mu + s^2/2$ is normally distributed, the expected revenue $\exp(\mu + s^2/2)$ follows a log-normal distribution $P_{\exp(\mu + s^2/2)}(y) = Lognorm(y | \mu_N + s^2/2, \sigma_N^2)$. Thus, the distribution of $p\exp(\mu + s^2/2)$ is modeled as the product of a Beta and a log-normal distribution $P_{p\exp(\mu + s^2/2)} \sim \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s) Lognorm(y ; \mu_N + s^2/2, \sigma_N^2)$. In each experiment, data is added in increments of `n_samp_step`. The experiment stops when the probability that one group’s mean exceeds the other reaches `prob_stop`, or when the sample size hits `n_samp_max`. If `n_samp_step` is small, the proportion of correctly identified groups may fall short of `prob_stop` due to model inaccuracies or outliers. With a sufficiently large `n_samp_step`, the share of correctly identified groups aligns with the expected `prob_stop`.

$$
\begin{split}
P_{\text{users}}(x) & = 
\begin{cases}
1-p, \, x = 0
\\
p P_{\text{paying}}(x), \, x > 0
\end{cases}
= 
\begin{cases}
1-p, \, x = 0
\\
p Lognorm(x | s, \mu_N, \sigma_N), \, x > 0
\end{cases}
\\
E_{\text{users}}[x] & = p e^{\mu + s^2/2}
\\
P(p) & = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s),
\\
P_{\exp(\mu + s^2/2)}(y) & = Lognorm(y | \mu_N + s^2/2, \sigma_N^2)
\\
P_{p\exp(\mu + s^2/2)} & \sim \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s) Lognorm(y ; \mu_N + s^2/2, \sigma_N^2)
\end{split}
$$

In [ ]:
ConjugateRevPerUserParams = namedtuple('ConjugateRevPerUserParams', 'a b mu sigma sx')

def posterior_params_rev_per_user(data):
    d_paying = data[data != 0]
    d_paying_total = len(d_paying)
    d_total = len(data)
    a, b = posterior_params_binom(ns=d_paying_total, ntotal=d_total)
    post_pars = initial_params_lognormal(mu=np.log(d_paying)[0], sigma=np.std(np.log(d_paying)), sx=np.std(np.log(d_paying)))
    post_pars = posterior_params_lognormal(d_paying[1:], post_pars)
    return ConjugateRevPerUserParams(a=a, b=b, mu=post_pars.mu, sigma=post_pars.sigma, sx=post_pars.sx)

def posterior_params_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns
    return a, b

def rev_per_user_p_dist(params):
    return stats.beta(a=params.a, b=params.b)

def posterior_mean_rev_per_user_rvs(params, nsamples=100_000):
    p_dist = rev_per_user_p_dist(params)
    ps = p_dist.rvs(size=nsamples)
    means_dist = posterior_mean_dist_lognormal(params)
    means = means_dist.rvs(nsamples)
    return ps * means

def exact_rev_per_user_rvs(p, mu, s, nsamples):
    conv = stats.bernoulli.rvs(p=p, size=nsamples)
    rev = stats.lognorm.rvs(s=s, scale=np.exp(mu), size=nsamples)
    return conv * rev

def prob_pb_gt_pa_samples(post_samp_A, post_samp_B):
    if len(post_samp_A) != len(post_samp_B):
        return None
    b_gt_a = np.sum(post_samp_B > post_samp_A)
    return b_gt_a / len(post_samp_A)

nexps = 100
prob_stop = 0.95
n_samp_max = 3_000_000
n_samp_step = 30000
n_post_samp = 50000

A = {'p': 0.1, 'mu': 2, 's': 1}

cmp = pd.DataFrame(columns=['A_pars', 'B_pars', 'A_mean', 'B_mean', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])
cmp['A_pars'] = [A] * nexps
cmp['B_pars'] = cmp['A_pars'].apply(lambda x: {'p': x['p'] * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1)), 's': x['s'], 'mu': x['mu'] * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1))})
cmp['A_mean'] = cmp['A_pars'].apply(lambda x: x['p'] * stats.lognorm(s=x['s'], scale=np.exp(x['mu'])).mean())
cmp['B_mean'] = cmp['B_pars'].apply(lambda x: x['p'] * stats.lognorm(s=x['s'], scale=np.exp(x['mu'])).mean())
cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B_mean'] > r['A_mean'] else 'A', axis=1)

for i in range(nexps):
    A_pars = cmp.at[i, 'A_pars']
    B_pars = cmp.at[i, 'B_pars']
    n_samp_total = 0
    dA = np.array([])
    dB = np.array([])
    while n_samp_total < n_samp_max:
        dA = np.append(dA, exact_rev_per_user_rvs(p=A_pars['p'], mu=A_pars['mu'], s=A_pars['s'], nsamples=n_samp_step))
        dB = np.append(dB, exact_rev_per_user_rvs(p=B_pars['p'], mu=B_pars['mu'], s=B_pars['s'], nsamples=n_samp_step))
        n_samp_total += n_samp_step
        post_pars_A = posterior_params_rev_per_user(dA)
        post_pars_B = posterior_params_rev_per_user(dB)
        post_samp_A = posterior_mean_rev_per_user_rvs(post_pars_A)
        post_samp_B = posterior_mean_rev_per_user_rvs(post_pars_B)
        pb_gt_pa = prob_pb_gt_pa_samples(post_samp_A, post_samp_B)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_samp_A.mean()
            cmp.at[i, 'B_exp'] = post_samp_B.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: n_samp {n_samp_total}, best_group {best_gr}, P(b>a) {pb_gt_pa}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(10))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

# Orders per Visitor

A visitor may place several orders or none at all. The distribution of the number of orders per visitor, $P_{\text{orders}}(n)$, where $n \in 0, 1, 2, \dots$ can be modeled as a discrete analogue of the log-normal distribution or as a power-law distribution such as Zipf’s law: $P(n ; s) \propto n^{-s}$ [[ZipfDist](https://en.wikipedia.org/wiki/Zipf%27s_law#Formal_definition), [SciPyZipfian](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zipfian.html), [SciPyZipf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zipf.html)]. It is important to model the probabilities of low-order counts accurately. Zipf’s law may not provide sufficient flexibility for this. A more adaptable approach is to model the exact probabilities $p_i$ of making $i$ orders. Assume a maximum number of orders $N$, and let $n_i$ be the number of users with $i$ orders, for $i=0, 1, 2, \dots, N$. The likelihood is then given by the multinomial distribution: $P(\mathcal{D} | \mathcal{H}) = Mult(n_0, \dots, n_N | p_0, \dots, p_N)$ [[MultiDist](https://en.wikipedia.org/wiki/Multinomial_distribution), [SciPyMult](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multinomial.html)]. The conjugate prior for this likelihood is the Dirichlet distribution: $P(\mathcal{H}) = Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right)$ [[DirDist](https://en.wikipedia.org/wiki/Dirichlet_distribution), [SciPyDir](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.dirichlet.html)]. In the posterior distribution, each parameter is updated as $\alpha_i + n_i$. The marginal distributions of each $p_i$ are Beta distributions, consistent with interpreting $p_i$ as the conversion rate to exactly $i$ orders.

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = Mult(n_0, \dots, n_N | p_0, \dots, p_N) = \frac{(n_0 + \dots + n_N)!}{n_{0}! \dots n_{N}!} p_{0}^{n_{0}} \dots p_{N}^{n_{N}} 
\\
P(\mathcal{H}) & = 
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right) = 
\dfrac{1}{B( \alpha_{0}, \dots, \alpha_{N} )} \prod_{i=0}^{N} p_{i}^{\alpha_{i}-1},
\qquad
\sum_{i=0}^{N} p_i = 1,
\qquad
p_i \in [0, 1], 
\qquad
B(\alpha_{0}, \dots, \alpha_{N}) = 
\frac{\prod \limits_{i=0}^{N} \Gamma( \alpha_{i} )}
{\Gamma \left( \sum \limits_{i=0}^{N} \alpha_{i} \right)}
\\
P(\mathcal{H} | \mathcal{D}) 
& \propto Mult(n_0, \dots, n_N | p_0, \dots, p_N) Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right)
\\
& \propto
p_{0}^{n_{0}} \dots p_{N}^{n_{N}} 
\prod _{i=0}^{N} p_{i}^{\alpha_{i}-1}
\\
& \propto
\prod_{i=0}^{N} p_{i}^{n_{i} + \alpha_{i} - 1}
\\
& =
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0} + n_0, \dots, \alpha_{N} + n_N \right)
\\
P(p_i | \mathcal{D} ) & = 
\int dp_0 \dots dp_{i-1}dp_{i+i} \dots dp_N P(\mathcal{H} | \mathcal{D}) 
=
Beta( p_i; \alpha_i + n_i, \sum_{k=0}^{N} (\alpha_k + n_k) - \alpha_i - n_i )
\end{split}
$$

To illustrate parameter estimation, we define a Zipf distribution with parameters `s` and `Nmax`. A sample is drawn from this distribution, and the posterior distribution is then constructed based on the sample. In addition, the conversion probabilities for exactly $i$ orders, $p_i$, are computed. The plot shows the original distribution, the observed sample, the posterior predictive distribution $x$, the estimated values of $p_i$, and their 95\% credible intervals. For most values of $i$, the true probabilities fall within the corresponding credible intervals.

In [ ]:
def initial_params_dir(N):
    return np.ones(N)

def posterior_params_dir(data, initial_pars):
    u, c = np.unique(data, return_counts=True)
    post_pars = np.copy(initial_pars)
    for k, v in zip(u, c):
        post_pars[k] = post_pars[k] + v
    return post_pars

def posterior_dist_dir(params):
    return stats.dirichlet(alpha=params)

def posterior_nords_dir_rvs(params, nsamp):
    nords = np.empty(nsamp)
    d = posterior_dist_dir(params)
    probs = d.rvs(size=nsamp)
    for i, p in enumerate(probs):
        nords[i] = np.argmax(stats.multinomial.rvs(n=1, p=p))
    return nords

def marginal_pi_dist_dir(i, params):
    return stats.beta(a=params[i], b=np.sum(params) - params[i])

def posterior_pi_mean_95pdi(i, params):
    p = marginal_pi_dist_dir(i, params)
    m = p.mean()
    lower = p.ppf(0.025)
    upper = p.ppf(0.975)
    return m, lower, upper

Nmax = 30
s = 1.5
nsample = 1000

Npars = Nmax + 1
exact_dist = stats.zipfian(a=s, n=Npars, loc=-1)
data = exact_dist.rvs(nsample)
pars = initial_params_dir(Npars)
pars = posterior_params_dir(data, pars)
post_samp = posterior_nords_dir_rvs(pars, 100000)
pi = [posterior_pi_mean_95pdi(i, pars) for i in range(Npars)]

x = np.arange(0, Npars+1)
plt.figure()
plt.plot(x, exact_dist.pmf(x), color="black", label="Exact Zipf Distribution")
counts_data = np.bincount(data, minlength=len(x))
pmf_data = counts_data / counts_data.sum()
counts_post = np.bincount(post_samp.astype(int), minlength=len(x))
pmf_post = counts_post / counts_post.sum()
w = 0.25
plt.bar(x - w/2, pmf_data, width=w, color="black", label="Sample")
plt.bar(x + w/2, pmf_post, width=w, color="black", alpha=0.2, label=r"Posterior $n_i$")
y = [p[0] for p in pi]
yerr_lower = [p[0] - p[1] for p in pi]
yerr_upper = [p[2] - p[0] for p in pi]
plt.errorbar(x[:-1], y, yerr=[yerr_lower, yerr_upper], fmt='.', color='red', alpha=0.8, label=r'$p_i$ estimates')
plt.xlabel("Orders")
plt.ylabel("Probability")
plt.xlim(-1, Nmax + 1)
plt.legend()
plt.show()


The distribution of order counts allows us to estimate the average number of orders $E[n] = \sum_{i=0}^N i, p_i$, the conversion to at least one order $1 - p_0$, and the conversion to two or more orders $1 - p_0 - p_1$. Below is an example comparing the expected number of orders $E[n]$. Two Zipf distributions are defined, where group B has a shape parameter `s` that is 5\% smaller than group A. The first chart shows the true distributions, exact means, and estimated $p_i$ values. For most values of $i$, the true $p_i$ falls within the estimated intervals. The second chart presents the posterior distribution of the average number of orders. Given the selected parameters, the probability that the mean of group B exceeds that of group A is $P(E[n]_B > E[n]_A) = 90\%$.

In [ ]:
def posterior_nords_mean_rvs(params, nsample):
    ns = np.arange(len(params))
    probs = stats.dirichlet.rvs(alpha=params, size=nsample)
    means = np.sum(ns * probs, axis=1)
    return means

def prob_pb_gt_pa_samples(post_samp_A, post_samp_B):
    if len(post_samp_A) != len(post_samp_B):
        return None
    b_gt_a = np.sum(post_samp_B > post_samp_A)
    return b_gt_a / len(post_samp_A)

nsample = 3000
Nmax = 30
Npars = Nmax + 1

post_samp_len = 100000
A, B = {}, {}
s = 1.5
A['dist_pars'] = {'s': s}
B['dist_pars'] = {'s': s * 0.95}
for g in [A, B]:
    g['exact_dist'] = stats.zipfian(a=g['dist_pars']['s'], n=Npars, loc=-1)
    g['data'] = g['exact_dist'].rvs(nsample)
    g['post_pars'] = initial_params_dir(Npars)
    g['post_pars'] = posterior_params_dir(g['data'], g['post_pars'])
    g['post_nords'] = posterior_nords_dir_rvs(g['post_pars'], post_samp_len)
    g['post_means'] = posterior_nords_mean_rvs(g['post_pars'], post_samp_len)
    g['pi'] = [posterior_pi_mean_95pdi(i, g['post_pars']) for i in range(Npars)]

x = np.arange(0, Npars)
plt.figure()
w = 0.35
plt.bar(x - w/2, A["exact_dist"].pmf(x), width=w, color="black", alpha=0.2, label="Exact distribution A")
plt.bar(x + w/2, B["exact_dist"].pmf(x), width=w, color="black", alpha=0.8, label="Exact distribution B")
plt.plot([A["exact_dist"].mean(), A["exact_dist"].mean()],[0, np.max(A["exact_dist"].pmf(x))*1.1],
         color="black", linestyle="dashed", alpha=0.3, label="Exact mean A")
plt.plot([B["exact_dist"].mean(), B["exact_dist"].mean()],[0, np.max(B["exact_dist"].pmf(x))*1.1],
         color="black", linestyle="dashed", label="Exact mean B")
yA = [p[0] for p in A["pi"]]
yAerr = [[p[0]-p[1] for p in A["pi"]],[p[2]-p[0] for p in A["pi"]]]
plt.errorbar(x - w/2, yA, yerr=yAerr, fmt=".", color="black", alpha=0.3, label=r"$p_i, A$")
yB = [p[0] for p in B["pi"]]
yBerr = [[p[0]-p[1] for p in B["pi"]],[p[2]-p[0] for p in B["pi"]]]
plt.errorbar(x + w/2, yB, yerr=yBerr, fmt=".", color="black", label=r"$p_i, B$")
plt.xlabel("Orders")
plt.ylabel("Probability")
plt.xlim(-1, Npars + 1 - 20)
plt.title("Orders per User")
plt.legend()
plt.show()

x = np.arange(0, Npars)
plt.figure()
bins = np.linspace(-0.5, Nmax + 0.5, 10*Nmax)
hA, _ = np.histogram(A['post_means'], bins=bins, density=True)
hB, _ = np.histogram(B['post_means'], bins=bins, density=True)
centers = (bins[:-1] + bins[1:])/2
w = (bins[1]-bins[0])/2
plt.bar(centers - w/2, hA, width=w, color='black', alpha=0.3, label=r'$E[n], A$')
plt.bar(centers + w/2, hB, width=w, color='black', alpha=1.0, label=r'$E[n], B$')
plt.plot([A['exact_dist'].mean(), A['exact_dist'].mean()], [0, np.max(hA)*1.1],
         color='black', linestyle='dashed', alpha=0.3, label='Exact mean A')
plt.plot([B['exact_dist'].mean(), B['exact_dist'].mean()], [0, np.max(hB)*1.1],
         color='black', linestyle='dashed', label='Exact mean B')
plt.xlabel('Orders')
plt.ylabel('Probability density')
plt.xlim(-1, Npars + 1 - 20)
plt.title('Average Order Number')
plt.legend()
plt.show()

print(f"P(E[n]_B > E[n]_A): {prob_pb_gt_pa_samples(A['post_means'], B['post_means'])}")

The number of correctly identified "better" groups is tested across `nexps` experiments. In Group A, the number of orders per user follows a Zipf distribution with parameter `s`, while in Group B the parameter varies within $\pm 5\%$ of A. Groups are compared based on their mean number of orders. In each experiment, samples are added incrementally by `n_samp_step`, posterior parameters are updated, and the probability $P(E[n]_B > E[n]_A)$ is computed. The experiment stops when the probability that one group’s mean exceeds the other’s reaches `prob_stop`, or when the maximum number of samples `n_samp_max` is reached. The proportion of correctly identified groups, 0.94, is close to the expected threshold `prob_stop = 0.95`.

In [ ]:
cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])

s = 1.5
Nmax = 30
Npars = Nmax + 1
nexps = 100
cmp['A'] = [s] * nexps
cmp['B'] = s * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))

n_samp_max = 200000
n_samp_step = 5000

prob_stop = 0.95
for i in range(nexps):
    s_a = cmp.at[i, 'A']
    s_b = cmp.at[i, 'B']
    exact_dist_a = stats.zipfian(a=s_a, n=Npars, loc=-1)
    exact_dist_b = stats.zipfian(a=s_b, n=Npars, loc=-1)
    cmp.at[i, 'best_exact'] = 'A' if exact_dist_a.mean() > exact_dist_b.mean() else 'B'
    n_samp_total = 0
    pars_a = initial_params_dir(Npars)
    pars_b = initial_params_dir(Npars)
    while n_samp_total < n_samp_max:
        data_a = exact_dist_a.rvs(n_samp_step)
        data_b = exact_dist_b.rvs(n_samp_step)
        n_samp_total += n_samp_step
        pars_a = posterior_params_dir(data_a, pars_a)
        pars_b = posterior_params_dir(data_b, pars_b)
        post_samp_len = 10000
        post_means_a = posterior_nords_mean_rvs(pars_a, post_samp_len)
        post_means_b = posterior_nords_mean_rvs(pars_b, post_samp_len)
        pb_gt_pa = prob_pb_gt_pa_samples(post_means_a, post_means_b)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_means_a.mean()
            cmp.at[i, 'B_exp'] = post_means_b.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: nsamp {n_samp_total}, best_gr {best_gr}, P(B>A) {pb_gt_pa}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(10))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

# Conclusion

Bayesian modeling has been applied to compare conversions, means using the central limit theorem, transactional revenue per user, and orders per visitor. For each metric, a model distribution was proposed. The model parameters are defined using conjugate prior distributions, enabling the analytical construction of posterior distributions. The process demonstrated parameter estimation from a sample, comparison of two groups, and the evaluation of the proportion of correctly identified "better" groups in a series of experiments. The proportion aligns with expectations. Model validation was not discussed, but in practice, it is crucial to assess the model's applicability to specific situations.

# References

[Apx] - [Bayesian A/B-Testing, Appendices](https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/appendices), *GitHub*.  
[BaseFal] - [Base Rate Fallacy](https://en.wikipedia.org/wiki/Base_rate_fallacy), *Wikipedia.*  
[BernProc] - [Bernoulli Process](https://en.wikipedia.org/wiki/Bernoulli_process), *Wikipedia.*  
[BerryEsseenTheorem] - [Berry-Esseen Theorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem), *Wikipedia.*   
[BetaDist] - [Beta Distribution](https://en.wikipedia.org/wiki/Beta_distribution), *Wikipedia.*     
[BinomDist] - [Binomial Distribution](https://en.wikipedia.org/wiki/Binomial_distribution), *Wikipedia.*  
[CausalDAG] - [Causal Graph](https://en.wikipedia.org/wiki/Causal_graph), *Wikipedia.*    
[CDF] - [Cumulative Distribution Function](https://en.wikipedia.org/wiki/Cumulative_distribution_function), *Wikipedia.*  
[CLT] - [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem), *Wikipedia.*    
[ConjPrior] - [Conjugate Prior](https://en.wikipedia.org/wiki/Conjugate_prior), *Wikipedia.*   
[DirDist] - [Dirichlet Distribution](https://en.wikipedia.org/wiki/Dirichlet_distribution), *Wikipedia.*    
[GammaDist] - [Gamma Distribution](https://en.wikipedia.org/wiki/Gamma_distribution), *Wikipedia.*     
[LognormDist] - [Log-normal Distribution](https://en.wikipedia.org/wiki/Log-normal_distribution), *Wikipedia.*    
[LomaxDist] - [Lomax Distribution](https://en.wikipedia.org/wiki/Lomax_distribution), *Wikipedia.*     
[MultiDist] - [Multinomial Distribution](https://en.wikipedia.org/wiki/Multinomial_distribution), *Wikipedia.*    
[NormDist] - [Normal Distribution](https://en.wikipedia.org/wiki/Normal_distribution), *Wikipedia.*  
[NormSum] - [Sum of Normally Distributed Random Variables](https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables), *Wikipedia.*  
[ParetoDist] - [Pareto Distribution](https://en.wikipedia.org/wiki/Pareto_distribution), *Wikipedia.*     
[ProbConv] - [Convolution of Probability Distributions](https://en.wikipedia.org/wiki/Convolution_of_probability_distributions), *Wikipedia.*    
[RandVarsConv] - [Convergence of Random Variables](https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_distribution), *Wikipedia.*   
[SGBS] - B. Lambert, A Student’s Guide to Bayesian Statistics ([Textbook](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [Student Resources](https://study.sagepub.com/lambert)).     
[SR] - R. McElreath, Statistical Rethinking: A Bayesian Course with Examples in R and STAN ([Textbook](https://www.routledge.com/Statistical-Rethinking-A-Bayesian-Course-with-Examples-in-R-and-STAN/McElreath/p/book/9780367139919), [Video Lectures](https://www.youtube.com/playlist?list=PLDcUM9US4XdPz-KxHM4XHt7uUVGWWVSus), [Course Materials](https://github.com/rmcelreath/stat_rethinking_2024)).   
[SciPyBern] - [scipy.stats.bernoulli](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bernoulli.html), *SciPy Reference.*  
[SciPyBeta] - [scipy.stats.beta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html), *SciPy Reference.*   
[SciPyBinom] - [scipy.stats.binom](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html), *SciPy Reference.*   
[SciPyDir] - [scipy.stats.dirichlet](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.dirichlet.html), *SciPy Reference.*  
[SciPyGamma] - [scipy.stats.gamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html), *SciPy Reference.*     
[SciPyLognorm] - [scipy.stats.lognorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html), *SciPy Reference.*      
[SciPyLomax] - [scipy.stats.lomax](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lomax.html), *SciPy Reference.*       
[SciPyMult] - [scipy.stats.multinomial](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multinomial.html), *SciPy Reference.*   
[SciPyNorm] - [scipy.stats.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html), *SciPy Reference.*   
[SciPyPareto] - [scipy.stats.pareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html), *SciPy Reference.*    
[SciPyZipf] - [scipy.stats.zipf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zipf.html), *SciPy Reference.*   
[SciPyZipfian] - [scipy.stats.zipfian](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zipfian.html), *SciPy Reference.*    
[SubjProb] - [Probability Interpretations](https://en.wikipedia.org/wiki/Probability_interpretations#Subjectivism), *Wikipedia.*    
[ZipfDist] - [Zipf's Law](https://en.wikipedia.org/wiki/Zipf%27s_law#Formal_definition), *Wikipedia.*   